# Feature Extract

Usage: function plotting will plot rmsd, rmsd histogram, rmsf and PCA projection

In [2]:
from __future__ import print_function
%matplotlib inline
import mdtraj as md
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import pandas as pd
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')
import MDAnalysis as mda
#XRD Ensemble
#28 4NPQ
#18 4HFI

In [3]:
residue_selection = 'resSeq 8 to 316'

In [4]:
traj_notes = ['5NJY_pH70_md1','5NJY_pH70_md2','5NJY_pH70_md3','5NJY_F238L_pH70_md1',
                 '5NJY_F238L_pH70_md2','5NJY_F238L_pH70_md3','5NJY_I233T_pH70_md1',
                  '5NJY_I233T_pH70_md2','5NJY_I233T_pH70_md3','5NJY_F238LI233T_pH70_md1',
                  '5NJY_F238LI233T_pH70_md2','5NJY_F238LI233T_pH70_md3','5NJY_pH46_md1','5NJY_pH46_md2','5NJY_pH46_md3','5NJY_F238L_pH46_md1',
                 '5NJY_F238L_pH46_md2','5NJY_F238L_pH46_md3','5NJY_I233T_pH46_md1',
                  '5NJY_I233T_pH46_md2','5NJY_I233T_pH46_md3','5NJY_F238LI233T_pH46_md1',
                  '5NJY_F238LI233T_pH46_md2','5NJY_F238LI233T_pH46_md3','4HFI_pH46_md1','4HFI_pH46_md2','4HFI_pH46_md3','4HFI_F238L_pH46_md1',
                 '4HFI_F238L_pH46_md2','4HFI_F238L_pH46_md3','4HFI_I233T_pH46_md1',
                  '4HFI_I233T_pH46_md2','4HFI_I233T_pH46_md3','4HFI_F238LI233T_pH46_md1',
                  '4HFI_F238LI233T_pH46_md2','4HFI_F238LI233T_pH46_md3','4NPQ_pH70_md5','4NPQ_pH70_md6','4NPQ_pH70_md7','4NPQ_F238L_pH70_md3',
                 '4NPQ_F238L_pH70_md4','4NPQ_F238L_pH70_md5','4NPQ_I233T_pH70_md3',
                  '4NPQ_I233T_pH70_md4','4NPQ_I233T_pH70_md5','4NPQ_F238LI233T_pH70_md3',
                  '4NPQ_F238LI233T_pH70_md4','4NPQ_F238LI233T_pH70_md5','4HFI_pH70_md1','4NPQ_F238LI233T_pH46_md1',
               '4NPQ_F238L_pH46_md1','4NPQ_F238L_pH46_md2','4NPQ_pH46_md1','4NPQ_I233T_pH46_md1','4NPQ_I233T_pH46_md2','4NPQ_I233T_pH46_md3']

In [5]:
def create_md_dataframe():    
    md_data = pd.DataFrame(columns=list(['MD_name','pH','replicate','traj_time']))
    return md_data

In [20]:
def create_metadata(md_data= None):   
    def append_metadata(traj_note,location = '/media/scottzhuang/data/MD/',skip=10,md_data= md_data):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        if skip == 1:
            traj = md.load(location + traj_location,top= location + top_location,stride=10)
        else:
            traj = md.load(location + traj_location,top= location + top_location)
        print("In " + traj_note + ", simulation runs " + str(10 * traj.n_frames) + " ns.")
        md_name = traj_note[:traj_note.find('pH')-1]
        pH = traj_note[traj_note.find('pH')+2:traj_note.find('pH')+4]
        md_replicate = traj_note[-1]
        for i in range(0,traj.n_frames):
            md_data.loc[md_data.shape[0]+1] = [md_name,pH,md_replicate,i]

    for traj_note in traj_notes:
        append_metadata(traj_note)

In [79]:
def create_system_notation(md_data = None):
    system_notation = 0
    notation = -1
    location = '/media/scottzhuang/data/MD/'
    skip=10
    notations = []
    increment = 0
    for traj_note in traj_notes:
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top= location + top_location)
        if traj_note.find('md1') >= 0:
            notation = notation + 1
        if traj_note == '4NPQ_pH70_md5' or  traj_note == '4NPQ_F238L_pH70_md3' or traj_note == '4NPQ_I233T_pH70_md3' or traj_note == '4NPQ_F238LI233T_pH70_md3':
            notation = notation + 1

        for frame in range(0,traj.n_frames):
            notations.append(notation)
        #if increment % 3 == 2:
        #    notation = notation + 1
        #increment = increment + 1 
        
    md_data['system'] = notations

In [80]:
create_system_notation(md_data)

In [8]:
def create_rmsd_data(md_data= None):
    def append_rmsd_data(traj_note,location = '/media/scottzhuang/data/MD/',ref_name = None, skip=10):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top= location + top_location)
        if ref_name != None:
            ref_location = ("/home/scottzhuang/masterthesis/miscellanies/pdb_ensemble/" + ref_name + ".pdb")
        else:
            ref_location = location + top_location
        ref_traj = md.load(ref_location)
        topology = traj.topology
        if traj.n_atoms != ref_traj.n_atoms:
            traj = traj.atom_slice(topology.select(residue_selection))
        traj.superpose(ref_traj,0)
        rmsd_data.extend(list(md.rmsd(traj, ref_traj)*10))
    rmsd_data = []
    for traj_note in traj_notes:
        append_rmsd_data(traj_note)
    md_data['rmsd']= rmsd_data

In [9]:
def create_cppca_data(md_data = None,residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
    def combined_ppca_reduced_cartesian(residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
        wholetraj = md.load("/home/scottzhuang/masterthesis/miscellanies/pdb_ensemble/new_ensemble.pdb")
        wholetraj.superpose(wholetraj,28)
        topology = wholetraj.topology
        wholetraj_sliced_ecd = wholetraj.atom_slice(topology.select(residue_selection_1))
        wholetraj_sliced_ecd.superpose(wholetraj_sliced_ecd,28)
        wholetraj_sliced_tmd = wholetraj.atom_slice(topology.select(residue_selection_2))
        wholetraj_sliced_tmd.superpose(wholetraj_sliced_tmd,28)
        ppca_ecd = PCA(n_components=1)
        ppca_tmd = PCA(n_components=1)
        partial_reduced_cartesian = [ppca_ecd.fit_transform(wholetraj_sliced_ecd.xyz.reshape(wholetraj_sliced_ecd.n_frames, wholetraj_sliced_ecd.n_atoms * 3)), ppca_tmd.fit_transform(wholetraj_sliced_tmd.xyz.reshape(wholetraj_sliced_tmd.n_frames, wholetraj_sliced_tmd.n_atoms * 3))]    
        return partial_reduced_cartesian, wholetraj_sliced_ecd, wholetraj_sliced_tmd,ppca_ecd,ppca_tmd
    
    def append_projection_on_combined_ppca_data(traj_note, location = '/media/scottzhuang/data/MD/',skip = 10, residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top= location + top_location)
        topology = traj.topology
        traj_sliced_ecd = traj.atom_slice(topology.select(residue_selection_1))
        traj_sliced_tmd = traj.atom_slice(topology.select(residue_selection_2))

        traj_sliced_ecd.superpose(wholetraj_sliced_ecd,28)
        traj_sliced_tmd.superpose(wholetraj_sliced_tmd,28)

        reduced_cartesian_ecd.extend(ppca_ecd.transform(traj_sliced_ecd.xyz.reshape(traj_sliced_ecd.n_frames, traj_sliced_ecd.n_atoms * 3)).T[0])
        reduced_cartesian_tmd.extend(ppca_tmd.transform(traj_sliced_tmd.xyz.reshape(traj_sliced_tmd.n_frames, traj_sliced_tmd.n_atoms * 3)).T[0])
    partial_reduced_cartesian, wholetraj_sliced_ecd,wholetraj_sliced_tmd, ppca_ecd,ppca_tmd = combined_ppca_reduced_cartesian(residue_selection_1,residue_selection_2)
    reduced_cartesian_ecd = []
    reduced_cartesian_tmd = []
    for traj_note in traj_notes:
        append_projection_on_combined_ppca_data(traj_note)
    md_data['ecd_pc1']= reduced_cartesian_ecd
    md_data['tmd_pc1']= reduced_cartesian_tmd

In [10]:
def create_cppca_data_2(md_data = None,residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
    def combined_ppca_reduced_cartesian(residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
        wholetraj = md.load("/home/scottzhuang/masterthesis/miscellanies/pdb_ensemble/new_ensemble.pdb")
        wholetraj.superpose(wholetraj,28)
        topology = wholetraj.topology
        wholetraj_sliced_ecd = wholetraj.atom_slice(topology.select(residue_selection_1))
        wholetraj_sliced_ecd.superpose(wholetraj_sliced_ecd,28)
        wholetraj_sliced_tmd = wholetraj.atom_slice(topology.select(residue_selection_2))
        wholetraj_sliced_tmd.superpose(wholetraj_sliced_tmd,28)
        ppca_ecd = PCA(n_components=2)
        ppca_tmd = PCA(n_components=2)
        partial_reduced_cartesian = [ppca_ecd.fit_transform(wholetraj_sliced_ecd.xyz.reshape(wholetraj_sliced_ecd.n_frames, wholetraj_sliced_ecd.n_atoms * 3)), ppca_tmd.fit_transform(wholetraj_sliced_tmd.xyz.reshape(wholetraj_sliced_tmd.n_frames, wholetraj_sliced_tmd.n_atoms * 3))]    
        return partial_reduced_cartesian, wholetraj_sliced_ecd, wholetraj_sliced_tmd,ppca_ecd,ppca_tmd
    
    def append_projection_on_combined_ppca_data(traj_note, location = '/media/scottzhuang/data/MD/',skip = 10, residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top= location + top_location)
        topology = traj.topology
        traj_sliced_ecd = traj.atom_slice(topology.select(residue_selection_1))
        traj_sliced_tmd = traj.atom_slice(topology.select(residue_selection_2))

        traj_sliced_ecd.superpose(wholetraj_sliced_ecd,28)
        traj_sliced_tmd.superpose(wholetraj_sliced_tmd,28)

        reduced_cartesian_ecd.extend(ppca_ecd.transform(traj_sliced_ecd.xyz.reshape(traj_sliced_ecd.n_frames, traj_sliced_ecd.n_atoms * 3)).T[1])
        reduced_cartesian_tmd.extend(ppca_tmd.transform(traj_sliced_tmd.xyz.reshape(traj_sliced_tmd.n_frames, traj_sliced_tmd.n_atoms * 3)).T[1])
    partial_reduced_cartesian, wholetraj_sliced_ecd,wholetraj_sliced_tmd, ppca_ecd,ppca_tmd = combined_ppca_reduced_cartesian(residue_selection_1,residue_selection_2)
    reduced_cartesian_ecd = []
    reduced_cartesian_tmd = []
    for traj_note in traj_notes:
        append_projection_on_combined_ppca_data(traj_note)
    md_data['ecd_pc2']= reduced_cartesian_ecd
    md_data['tmd_pc2']= reduced_cartesian_tmd

In [ ]:
def create_pca_data(md_data = None):
    def ensemble_pca_cartesian():
        wholetraj = md.load("/home/scottzhuang/masterthesis/miscellanies/pdb_ensemble/new_ensemble.pdb")
        wholetraj.superpose(wholetraj,28)
        pca = PCA(n_components=2)
        reduced_cartesian = pca.fit_transform(wholetraj.xyz.reshape(wholetraj.n_frames,wholetraj.n_atoms *3))
        return wholetraj, pca
    def append_pca_data(traj_note,location = '/media/scottzhuang/data/MD/',skip = 10):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top =location + top_location)
        topology = traj.topology
        traj = traj.atom_slice(topology.select("resSeq 8 to 316"))
        traj.superpose(wholetraj,28)
        traj_reduced_cartesian = pca.transform(traj.xyz.reshape(traj.n_frames,traj.n_atoms * 3))
        pca1.extend(traj_reduced_cartesian.T[0])
        pca2.extend(traj_reduced_cartesian.T[1])
    pca1 = []
    pca2 = []
    wholetraj, pca = ensemble_pca_cartesian()
    for traj_note in traj_notes:
        append_pca_data(traj_note)
    md_data['wholepca_pc1'] = pca1
    md_data['wholepca_pc2'] = pca2

In [12]:
def create_domain_twist_data(md_data = None):
    def append_domain_twist(traj_note,location = '/media/scottzhuang/data/MD/',skip = 10):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top= location + top_location)        
        traj.superpose(traj,0)
        topology = traj.topology
        domain_twist = []
        for chain in range (0,5):
            residue_selection_1 = "resid " + str(8+chain*311) + " to " + str(192+chain*311)
            residue_selection_2 = "resid " + str(192+chain*311) + " to " + str(310+chain*311)
            traj_sliced_ecd = traj.atom_slice(topology.select(residue_selection_1))
            traj_sliced_tmd = traj.atom_slice(topology.select(residue_selection_2))
            angle = []
            for i in range(0,traj.n_frames):
                cen_mass_ecd = md.compute_center_of_mass(traj_sliced_ecd[i])[0]
                cen_mass_tmd = md.compute_center_of_mass(traj_sliced_tmd[i])[0]
                cen_mass = md.compute_center_of_mass(traj[i])[0]
                cen_mass_ecd[2] = cen_mass[2]
                cen_mass_tmd[2] = cen_mass[2]
                vec_ecd = cen_mass_ecd - cen_mass
                vec_tmd = cen_mass_tmd - cen_mass
                veclength_ecd = np.sqrt(np.sum(np.power(vec_ecd,2)))
                veclength_tmd = np.sqrt(np.sum(np.power(vec_tmd,2)))
                angle.append(57.2958 * np.arccos(np.dot(vec_ecd,vec_tmd) /(veclength_ecd * veclength_tmd)))
            domain_twist.append(angle)
        domain_twist_data.extend(np.mean(np.asarray(domain_twist),axis=0))
    domain_twist_data = []
    for traj_note in traj_notes:
        domain_twist_avg = append_domain_twist(traj_note)
    md_data['domain twist']= domain_twist_data

In [13]:
def create_hole_data(md_data = None):
    import MDAnalysis as mda
    from MDAnalysis.analysis.hole import HOLEtraj
    def append_hole_data(traj_note,location = '/media/scottzhuang/data/MD/',skip = 10): 
        hole_traj_location = location + traj_note + '/' + traj_note + ".hole.traj.pdb"
        u = mda.Universe(hole_traj_location)
        H = HOLEtraj(u, executable="~/hole2/exe/hole")
        H.run()
        hole_data.extend(H.min_radius().T[1])
    hole_data = []
    for traj_note in traj_notes:
        append_hole_data(traj_note)
    md_data['min hole radius']=hole_data

In [14]:
def create_hbond_data(md_data = None):
    import MDAnalysis as mda
    from MDAnalysis.analysis.hbonds import HydrogenBondAnalysis
    def append_hbond_data(traj_note,location = '/media/scottzhuang/data/MD/', skip=10):
        top_location = traj_note + '/' + traj_note + ".protein.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".protein.xtc"
        traj = mda.Universe(location + top_location,location + traj_location)
        h = HydrogenBondAnalysis(traj,'protein','protein',distance=3.0, angle=120.0)
        h.run()
        hbond_data.extend(h.count_by_time().T[1])
    hbond_data = []
    for traj_note in traj_notes:
        append_hbond_data(traj_note)
    md_data['h_bond number'] = hbond_data

In [15]:
def create_helix_tilt_data(md_data = None):
    def append_helix_tilt_data(traj_note,location = '/media/scottzhuang/data/MD/', skip=10):
        tilt_data = pd.read_csv(location + traj_note + '/' + traj_note + '.tilt.csv',sep=" ")
        tilt_data.columns = ['traj_time','avg','ang1','ang2','ang3','ang4','ang5']
        helix_tilt_data.extend(tilt_data['avg'])
    helix_tilt_data = []
    for traj_note in traj_notes:
        append_helix_tilt_data(traj_note)
    md_data['helix tilt angle'] = helix_tilt_data

In [16]:
def create_helix_twist_data(md_data = None):
    def append_helix_twist_data(traj_note,location = '/media/scottzhuang/data/MD/', skip=10):
        twist = pd.read_csv(location + traj_note + '/' + traj_note + '.twist.csv',sep=" ")
        twist.columns = ['traj_time','avg','ang1','ang2','ang3','ang4','ang5']
        helix_twist_data.extend(twist['avg'])
    helix_twist_data = []
    for traj_note in traj_notes:
        append_helix_twist_data(traj_note)
    md_data['helix twist angle'] = helix_twist_data

In [27]:
def create_pore_profile(md_data = None):
    def pore_rad(traj_note,location = '/media/scottzhuang/data/MD/'):
        top_location = traj_note + '/' + traj_note + ".protein.gro"
        traj_name = traj_note + '/' + traj_note +  ".skip10.protein.xtc"    
        traj = md.load(location + traj_name,top = location +top_location)
        ca_top_location = traj_note + '/' + traj_note +  ".ca.pdb"
        ca_traj_name = traj_note + '/' + traj_note +  ".skip10.ca.xtc"    
        ca_traj = md.load(location + ca_traj_name,top = location + ca_top_location)
        topology = traj.topology
        pore_profile = pd.DataFrame(columns=['traj_time','resid','pore_radius'])
        m = 0
        incre = 0
        for i in [217,221,225,228,232,235]:
            group_1 = np.arange(i,i + 1245, 311)
            xyz_inter = np.mean(ca_traj.xyz[:,group_1],axis=1)
            chain = topology.add_chain()
            residue = topology.add_residue('ALA',chain)
            topology.add_atom('H','H',residue)
            traj.topology = topology
            traj.xyz = np.append(traj.xyz,xyz_inter.reshape([traj.n_frames,1,3]),axis=1)
            pairs = list(itertools.product(group_1,[1555 + incre]))
            for j in range(0,traj.n_frames):
                pore_profile.loc[m] = [j,i,np.mean(md.compute_contacts(traj[j], pairs)[0])]
                m = m + 1
            incre = incre +1
        pore_data_222.extend(pore_profile[pore_profile['resid'] == 217]['pore_radius'])
        pore_data_226.extend(pore_profile[pore_profile['resid'] == 221]['pore_radius'])
        pore_data_230.extend(pore_profile[pore_profile['resid'] == 225]['pore_radius'])
        pore_data_233.extend(pore_profile[pore_profile['resid'] == 228]['pore_radius'])
        pore_data_237.extend(pore_profile[pore_profile['resid'] == 232]['pore_radius'])
        pore_data_240.extend(pore_profile[pore_profile['resid'] == 235]['pore_radius'])
    import itertools
    pore_data_222 = []
    pore_data_226 = []
    pore_data_230 = []
    pore_data_233 = []
    pore_data_237 = []
    pore_data_240 = []

    for traj_note in traj_notes:
        pore_rad(traj_note)
    md_data['pore_profile_222'] = pore_data_222
    md_data['pore_profile_226'] = pore_data_226
    md_data['pore_profile_230'] = pore_data_230
    md_data['pore_profile_233'] = pore_data_233
    md_data['pore_profile_237'] = pore_data_237
    md_data['pore_profile_240'] = pore_data_240

In [56]:
def create_hydration_profile(md_data = None):
    def append_hydration_data(traj_note,location = '/media/scottzhuang/data/MD/', skip=10):
        top_location = traj_note + '/' + traj_note + ".system.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".system.xtc"
        traj = mda.Universe(location + top_location,location + traj_location)
        hydration = traj.select_atoms("(cyzone 7 10 -10 resid 230) and name OW",updating = True)
        for i in range(0,traj.trajectory.n_frames):
            traj.trajectory[i]
            hydration_data.append(hydration.n_atoms)
    hydration_data = []
    for traj_note in traj_notes:
        append_hydration_data(traj_note)
    md_data['hydration_data'] = hydration_data

In [145]:
def create_beta_expansion(md_data = None):
    def append_beta_expansion_data(traj_note,location = '/media/scottzhuang/data/MD/', skip=10):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top = location + top_location)
        distance = []
        for chain in range(0,5):
            distance.append(md.compute_distances(traj,[[27 + chain * 311,185 + chain * 311]]))
        beta_expansion_data.extend(np.mean(distance,axis=0).T[0])
    beta_expansion_data = []
    for traj_note in traj_notes:
        append_beta_expansion_data(traj_note)
    md_data['beta_expansion'] = beta_expansion_data

In [139]:
def create_M2_radius(md_data = None):
    def append_M2_radius_data(traj_note,location = '/media/scottzhuang/data/MD/', skip=10):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top = location + top_location)
        topology = traj.topology
        M2_selection = 'resSeq 231 to 245'
        traj_M2 = traj.atom_slice(topology.select(M2_selection))
        M2_radius_data.extend(md.compute_rg(traj_M2).T) 
    M2_radius_data = []
    for traj_note in traj_notes:
        append_M2_radius_data(traj_note)
    md_data['M2_radius'] = M2_radius_data

In [143]:
def create_ECD_radius(md_data = None):
    def append_ECD_radius_data(traj_note,location = '/media/scottzhuang/data/MD/', skip=10):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top = location + top_location)
        topology = traj.topology
        ECD_selection = 'resSeq 5 to 194'
        traj_ECD = traj.atom_slice(topology.select(ECD_selection))
        ECD_radius_data.extend(md.compute_rg(traj_ECD).T) 
    ECD_radius_data = []
    for traj_note in traj_notes:
        append_ECD_radius_data(traj_note)
    md_data['ECD_radius'] = ECD_radius_data

In [6]:
#md_data = create_md_dataframe()
#create_metadata(md_data= md_data)
md_data = pd.read_csv("glic_gating.csv")

In [21]:
md_data = create_md_dataframe()
create_metadata(md_data= md_data)
create_system_notation(md_data = md_data)
create_rmsd_data(md_data= md_data)

In 5NJY_pH70_md1, simulation runs 1030 ns.
In 5NJY_pH70_md2, simulation runs 750 ns.
In 5NJY_pH70_md3, simulation runs 810 ns.
In 5NJY_F238L_pH70_md1, simulation runs 540 ns.
In 5NJY_F238L_pH70_md2, simulation runs 680 ns.
In 5NJY_F238L_pH70_md3, simulation runs 720 ns.
In 5NJY_I233T_pH70_md1, simulation runs 620 ns.
In 5NJY_I233T_pH70_md2, simulation runs 590 ns.
In 5NJY_I233T_pH70_md3, simulation runs 820 ns.
In 5NJY_F238LI233T_pH70_md1, simulation runs 830 ns.
In 5NJY_F238LI233T_pH70_md2, simulation runs 800 ns.
In 5NJY_F238LI233T_pH70_md3, simulation runs 600 ns.
In 5NJY_pH46_md1, simulation runs 550 ns.
In 5NJY_pH46_md2, simulation runs 800 ns.
In 5NJY_pH46_md3, simulation runs 560 ns.
In 5NJY_F238L_pH46_md1, simulation runs 930 ns.
In 5NJY_F238L_pH46_md2, simulation runs 950 ns.
In 5NJY_F238L_pH46_md3, simulation runs 1090 ns.
In 5NJY_I233T_pH46_md1, simulation runs 810 ns.
In 5NJY_I233T_pH46_md2, simulation runs 870 ns.
In 5NJY_I233T_pH46_md3, simulation runs 980 ns.
In 5NJY_F23

In [ ]:
create_pca_data(md_data = md_data)
create_cppca_data(md_data= md_data)
create_domain_twist_data(md_data= md_data)
#create_hole_data(md_data= md_data)
create_cppca_data_2(md_data)
#create_hbond_data(md_data=md_data)
create_pore_profile(md_data)
create_helix_tilt_data(md_data)
create_helix_twist_data(md_data)
create_beta_expansion(md_data)
create_M2_radius(md_data)
create_ECD_radius(md_data)

In [57]:
create_hydration_profile(md_data)

In [149]:
md_data.to_csv("glic_gating_new.csv")

In [147]:
md_data.columns

Index(['MD_name', 'pH', 'replicate', 'traj_time', 'system', 'rmsd',
       'wholepca_pc1', 'wholepca_pc2', 'ecd_pc1', 'tmd_pc1', 'domain twist',
       'ecd_pc2', 'tmd_pc2', 'pore_profile_222', 'pore_profile_226',
       'pore_profile_230', 'pore_profile_233', 'pore_profile_237',
       'pore_profile_240', 'helix tilt angle', 'helix twist angle',
       'hydration_data', 'beta_expansion', 'M2_radius', 'ECD_radius'],
      dtype='object')

In [148]:
md_data

,MD_name,pH,replicate,traj_time,system,rmsd,wholepca_pc1,wholepca_pc2,ecd_pc1,tmd_pc1,...,pore_profile_230,pore_profile_233,pore_profile_237,pore_profile_240,helix tilt angle,helix twist angle,hydration_data,beta_expansion,M2_radius,ECD_radius
1,5NJY,70,1,0,0,0.000000,0.717672,3.350286,-0.450258,3.279069,...,0.500961,0.333015,0.459541,0.369191,6.369974,1.140340,13,1.335429,1.113488,3.022238
2,5NJY,70,1,1,0,1.727738,0.630747,4.148448,0.894403,3.166664,...,0.485957,0.343227,0.516545,0.395003,6.165242,1.579362,21,1.360500,1.120347,3.080599
3,5NJY,70,1,2,0,2.020854,1.855653,3.210544,2.341526,2.749989,...,0.514719,0.352873,0.497097,0.388581,5.918166,1.862192,16,1.371708,1.122901,3.112705
4,5NJY,70,1,3,0,1.915493,1.412525,3.407094,1.985111,3.009501,...,0.531552,0.354509,0.478344,0.413634,6.423780,1.926211,18,1.332638,1.124136,3.104171
5,5NJY,70,1,4,0,1.899955,2.364887,3.248359,2.116412,3.058160,...,0.478727,0.325887,0.505637,0.408311,6.001121,0.773120,14,1.307941,1.114801,3.105131
6,5NJY,70,1,5,0,2.043537,1.866259,2.947304,2.543423,2.773529,...,0.492858,0.342843,0.497926,0.376411,5.970423,1.092484,16,1.312371,1.117528,3.122921
7,5NJY,70,1,6,0,1.885621,2.240278,2.618281,2.745998,2.625010,...,0.536927,0.373507,0.494826,0.360884,6.606431,2.170445,20,1.331622,1.118296,3.105504
8,5NJY,70,1,7,0,1.995083,2.609149,2.053409,3.198771,2.811840,...,0.522523,0.368475,0.500181,0.378251,6.841249,1.869606,21,1.313808,1.125552,3.118326
9,5NJY,70,1,8,0,2.042677,3.700861,2.066899,3.669585,2.774023,...,0.496173,0.375582,0.488795,0.389509,6.105285,2.865439,20,1.313913,1.127648,3.124871
10,5NJY,70,1,9,0,2.152671,3.595903,2.555983,3.925136,3.081736,...,0.517023,0.372421,0.500309,0.382157,6.414548,2.115714,17,1.345914,1.118554,3.135543
